# IDAES Skeleton Unit Model

This notebook demonstrates usage of the IDAES Skeleton Unit Model, which provides a generic "bares bones" unit for user-defined models and custom variable and constraint sets. To allow maximum versatility, this unit may be defined as a surrogate model or a custom equation-oriented model. Users must add ports and variables that match connected models, and this is facilitated through a provided method to add port-variable sets.

For users who wish to train surrogates with IDAES tools and insert obtained models into a flowsheet, see the more versatile `SurrogateBlock()` for streamlined integration.

We will begin with relevant imports. We will need basic Pyomo and IDAES components:

In [1]:
import pytest
from pyomo.environ import (check_optimal_termination,
                           ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Var,
                           Set,
                           value,
                           units as pyunits)
from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import SkeletonUnitModel
from idaes.core.util.model_statistics import degrees_of_freedom
from pyomo.util.check_units import assert_units_consistent

    idaes.models.unit_models  (deprecated in 2.0.0.alpha0) (called from
    <frozen importlib._bootstrap>:219)


For demonstrative purposes, we will build a simple model manually defining state variables relations for two generic components. In this model, the variables specify an FcTP system where molar flow of each componet, temperature and pressure are selected as state variables. Users can use this structure to write custom relations between inlet and outlet streams; for example, defining outlet pressure using an explicit correlation or outlet flow of a specific component from a known accumulation or loss in the system.

In [2]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.skeleton = SkeletonUnitModel(default={"dynamic": False})
m.fs.skeleton.comp_list = Set(initialize=["c1", "c2"])  # two generic components

# input vars for skeleton
# m.fs.time is a pre-initialized Set belonging to the FlowsheetBlock; for dynamic=False, time=[0]
m.fs.skeleton.flow_comp_in = Var(m.fs.time, m.fs.skeleton.comp_list, initialize=1.0, units=pyunits.mol/pyunits.s)
m.fs.skeleton.temperature_in = Var(m.fs.time, initialize=298.15, units=pyunits.K)
m.fs.skeleton.pressure_in = Var(m.fs.time, initialize=101, units=pyunits.kPa)

# output vars for skeleton
m.fs.skeleton.flow_comp_out = Var(m.fs.time, m.fs.skeleton.comp_list, initialize=1.0, units=pyunits.mol/pyunits.s)
m.fs.skeleton.temperature_out = Var(m.fs.time, initialize=298.15, units=pyunits.K)
m.fs.skeleton.pressure_out = Var(m.fs.time, initialize=101, units=pyunits.kPa)

# Surrogate model equations

# flow equation
def rule_flow(m, t, i):
    return m.flow_comp_out[t, i] == m.flow_comp_in[t, i]
m.fs.skeleton.eq_flow = Constraint(m.fs.time, m.fs.skeleton.comp_list, rule=rule_flow)

# note that all terms need to have consistent units - the "10" terms could be defined as separate variables with units
m.fs.skeleton.eq_temperature = Constraint(expr=m.fs.skeleton.temperature_out[0]
                                          == m.fs.skeleton.temperature_in[0] + 10*pyunits.K)
m.fs.skeleton.eq_pressure = Constraint(expr=m.fs.skeleton.pressure_out[0]
                                       == m.fs.skeleton.pressure_in[0] - 10*pyunits.kPa)

# dictionaries relating state properties to custom variables
inlet_dict = {"flow_mol_comp": m.fs.skeleton.flow_comp_in,
              "temperature": m.fs.skeleton.temperature_in,
              "pressure": m.fs.skeleton.pressure_in}
outlet_dict = {"flow_mol_comp": m.fs.skeleton.flow_comp_out,
               "temperature": m.fs.skeleton.temperature_out,
               "pressure": m.fs.skeleton.pressure_out,}

m.fs.skeleton.add_ports(name="inlet", member_dict=inlet_dict)
m.fs.skeleton.add_ports(name="outlet", member_dict=outlet_dict)

Let's see how many degrees of freedom the flowsheet has:

In [3]:
print(degrees_of_freedom(m))

4


By specifying four input values, we obtain a square problem and can initialize and solve the flowsheet similar to other IDAES unit models. While IDAES largely restricts users to consistent units throughout an entire model to satisfy unit model constraints, the Skeleton model allows users to only include a few specific equations and easily allows custom units as well. As shown below, the flowsheet requires only that all pressures be in kPa rather than global usage of standard base units of Pa.

In [4]:
m.fs.skeleton.inlet.flow_mol_comp[0, "c1"].fix(2*pyunits.mol/pyunits.s)
m.fs.skeleton.inlet.flow_mol_comp[0, "c2"].fix(2*pyunits.mol/pyunits.s)
m.fs.skeleton.inlet.temperature.fix(325*pyunits.K)
m.fs.skeleton.inlet.pressure.fix(200*pyunits.kPa)

assert degrees_of_freedom(m) == 0

m.fs.skeleton.initialize()

solver = SolverFactory('ipopt')
results = solver.solve(m, tee = True)

assert check_optimal_termination(results)
assert_units_consistent(m)

assert value(m.fs.skeleton.outlet.flow_mol_comp[0, "c1"]) == pytest.approx(2, abs=1e-3)
assert value(m.fs.skeleton.outlet.flow_mol_comp[0, "c2"]) == pytest.approx(2, abs=1e-3)
assert value(m.fs.skeleton.outlet.temperature[0]) == pytest.approx(335, abs=1e-3)
assert value(m.fs.skeleton.outlet.pressure[0]) == pytest.approx(190, abs=1e-3)

2022-04-13 11:31:28 [INFO] idaes.init.fs.skeleton: Initialization completed using default method optimal - Optimal Solution Found.
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain th

The Skeleton model allows for a custom, user-defined initialization scheme as well, as demonstrated below. Users can similarly introduce custom variable and constraint scaling using IDAES scaling tools (not demonstrated in this example):

In [5]:
# rebuild the flowsheet
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.skeleton = SkeletonUnitModel(default={"dynamic": False})
m.fs.skeleton.comp_list = Set(initialize=["c1", "c2"])  # two generic components

# input vars for skeleton
# m.fs.time is a pre-initialized Set belonging to the FlowsheetBlock; for dynamic=False, time=[0]
m.fs.skeleton.flow_comp_in = Var(m.fs.time, m.fs.skeleton.comp_list, initialize=1.0, units=pyunits.mol/pyunits.s)
m.fs.skeleton.temperature_in = Var(m.fs.time, initialize=298.15, units=pyunits.K)
m.fs.skeleton.pressure_in = Var(m.fs.time, initialize=101, units=pyunits.kPa)

# output vars for skeleton
m.fs.skeleton.flow_comp_out = Var(m.fs.time, m.fs.skeleton.comp_list, initialize=1.0, units=pyunits.mol/pyunits.s)
m.fs.skeleton.temperature_out = Var(m.fs.time, initialize=298.15, units=pyunits.K)
m.fs.skeleton.pressure_out = Var(m.fs.time, initialize=101, units=pyunits.kPa)

# Surrogate model equations

# flow equation
def rule_flow(m, t, i):
    return m.flow_comp_out[t, i] == m.flow_comp_in[t, i]
m.fs.skeleton.eq_flow = Constraint(m.fs.time, m.fs.skeleton.comp_list, rule=rule_flow)

# note that all terms need to have consistent units - the "10" terms could be defined as separate variables with units
m.fs.skeleton.eq_temperature = Constraint(expr=m.fs.skeleton.temperature_out[0]
                                          == m.fs.skeleton.temperature_in[0] + 10*pyunits.K)
m.fs.skeleton.eq_pressure = Constraint(expr=m.fs.skeleton.pressure_out[0]
                                       == m.fs.skeleton.pressure_in[0] - 10*pyunits.kPa)

# dictionaries relating state properties to custom variables
inlet_dict = {"flow_mol_comp": m.fs.skeleton.flow_comp_in,
              "temperature": m.fs.skeleton.temperature_in,
              "pressure": m.fs.skeleton.pressure_in}
outlet_dict = {"flow_mol_comp": m.fs.skeleton.flow_comp_out,
               "temperature": m.fs.skeleton.temperature_out,
               "pressure": m.fs.skeleton.pressure_out,}

m.fs.skeleton.add_ports(name="inlet", member_dict=inlet_dict)
m.fs.skeleton.add_ports(name="outlet", member_dict=outlet_dict)

# custom initialization scheme
m.fs.skeleton.inlet.flow_mol_comp[0, "c1"].fix(2*pyunits.mol/pyunits.s)
m.fs.skeleton.inlet.flow_mol_comp[0, "c2"].fix(2*pyunits.mol/pyunits.s)
m.fs.skeleton.inlet.temperature.fix(325*pyunits.K)
m.fs.skeleton.inlet.pressure.fix(200*pyunits.kPa)

assert degrees_of_freedom(m) == 0

def my_initialize(unit, **kwargs):
    # Callback for user provided initialization sequence
    unit.eq_temperature.deactivate()
    unit.eq_pressure.deactivate()
    unit.outlet.temperature.fix(325*pyunits.K)
    unit.outlet.pressure.fix(200*pyunits.kPa)

    if degrees_of_freedom == 0:
        solver.solve(unit)

    unit.eq_temperature.activate()
    unit.eq_pressure.activate()
    unit.outlet.temperature.unfix()
    unit.outlet.pressure.unfix()

m.fs.skeleton.config.initializer = my_initialize
m.fs.skeleton.initialize()

solver = SolverFactory('ipopt')
results = solver.solve(m, tee = True)

assert check_optimal_termination(results)
assert_units_consistent(m)

assert value(m.fs.skeleton.outlet.flow_mol_comp[0, "c1"]) == pytest.approx(2, abs=1e-3)
assert value(m.fs.skeleton.outlet.flow_mol_comp[0, "c2"]) == pytest.approx(2, abs=1e-3)
assert value(m.fs.skeleton.outlet.temperature[0]) == pytest.approx(335, abs=1e-3)
assert value(m.fs.skeleton.outlet.pressure[0]) == pytest.approx(190, abs=1e-3)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://